In [8]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [9]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [10]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [11]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [12]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [13]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [14]:
## 
n='Input_A6_019' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A6_019
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.008959898672064832
test_rmse: 0.009268158740034048
test_r2: 0.5100386934034731
feature number: 3
train_rmse: 0.008588125041434042
test_rmse: 0.008836072602832428
test_r2: 0.5544423158207465
feature number: 4
train_rmse: 0.008388699755111566
test_rmse: 0.008690123016618872
test_r2: 0.5696823450471525
feature number: 5
train_rmse: 0.00821138383475383
test_rmse: 0.00848460915669759
test_r2: 0.5902412124594324
feature number: 6
train_rmse: 0.008166237464569483
test_rmse: 0.008546812695179463
test_r2: 0.5841624946509603
feature number: 7
train_rmse: 0.008057032538233545
test_rmse: 0.00857743889413933
test_r2: 0.5798759393245889
feature number: 8
train_rmse: 0.008049789316769158
test_rmse: 0.008625829714728984
test_r2: 0.5751934557048012
feature number: 9
train_rmse: 0.008043867120029063
test_rmse: 0.00867291113193728
test_r2: 0.5707341902964451
feature number: 10
train_rmse: 0.008026834365108572
test_rm

train_rmse: 0.007949973213592625
test_rmse: 0.008818852841265754
test_r2: 0.5554140173302918
feature number: 15
train_rmse: 0.00794247363140636
test_rmse: 0.00883881141097079
test_r2: 0.553195282933199
feature number: 16
train_rmse: 0.007921305277013697
test_rmse: 0.008839656348347382
test_r2: 0.5532800584774564
feature number: 17
train_rmse: 0.007913674840421044
test_rmse: 0.00885176787529369
test_r2: 0.5521123404941239
feature number: 18
train_rmse: 0.007907520142871393
test_rmse: 0.008893133438041906
test_r2: 0.5478232429488821
feature number: 19
train_rmse: 0.00789417210904831
test_rmse: 0.008909838989827138
test_r2: 0.5461379386155136
feature number: 20
train_rmse: 0.007876684409595183
test_rmse: 0.008966413834087805
test_r2: 0.5403091599793786
Test_rmse_min = 1.fea_num: 5  2.rmse: 0.00848460915669759
Test_r2_max = 1. fea_num: 5  2.r2: 0.5902412124594324
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.008959898672064832
test_rmse: 0.0092681587400

train_rmse: 0.008166237464569483
test_rmse: 0.008546812695179463
test_r2: 0.5841624946509603
feature number: 7
train_rmse: 0.008057032538233545
test_rmse: 0.00857743889413933
test_r2: 0.5798759393245889
feature number: 8
train_rmse: 0.008049789316769158
test_rmse: 0.008625829714728984
test_r2: 0.5751934557048012
feature number: 9
train_rmse: 0.008043867120029063
test_rmse: 0.00867291113193728
test_r2: 0.5707341902964451
feature number: 10
train_rmse: 0.008026834365108572
test_rmse: 0.008682588217662645
test_r2: 0.569823591453399
feature number: 11
train_rmse: 0.008022169510497173
test_rmse: 0.00869416088035527
test_r2: 0.5686163396751882
feature number: 12
train_rmse: 0.007987688958654315
test_rmse: 0.00876739839424921
test_r2: 0.5610795318500922
feature number: 13
train_rmse: 0.007964081842132861
test_rmse: 0.008760308390104972
test_r2: 0.5618075458670428
feature number: 14
train_rmse: 0.007949973213592625
test_rmse: 0.008818852841265754
test_r2: 0.5554140173302918
feature number: 15


# 可以修改第一層的神經元數量

In [15]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='sigmoid',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [16]:
fea_num=[5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.008883938605000844
[0.009644056810709069, 0.009835970706185136, 0.008420109857921675, 0.00889631280405298, 0.008786001657480195, 0.007942195895916356, 0.008728552044796714, 0.00823300387578424, 0.0092001575052664, 0.009153024891895676]


test_r2_ave: 0.5346719497844317
[0.5036871922793947, 0.41930253624749136, 0.6505729106183277, 0.5011829489575339, 0.5208346787206125, 0.613591581876717, 0.5465884217215886, 0.5975158620981199, 0.47609875460548656, 0.517344610719044]


feature number: 6
test_rmse_ave: 0.009043309087451042
[0.009117402556947676, 0.0094792308485491, 0.008230811050697588, 0.008453811165181553, 0.009468930120000238, 0.007186579786326145, 0.009939095127838108, 0.008909711360555801, 0.00910279264410507, 0.010544726214309125]


test_r2_ave: 0.5147098660420679
[0.5564135990763721, 0.4606611863052392, 0.6661077651351642, 0.5495710597908574, 0.44344941890916756, 0.6836193655569643, 0.4121021268217693, 0.5286327562586757, 0.4871289257662367, 0.35

In [17]:
fea_num=[5,6,7]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.008766011881634927
[0.00978133743911278, 0.009555781450812756, 0.008468324130258232, 0.008202153043297966, 0.007945265150538217, 0.007782180404088969, 0.00947490598823264, 0.008594702414274251, 0.008860566221813904, 0.008994902573919562]


test_r2_ave: 0.5476121683249254
[0.4894568592831481, 0.4519150296960992, 0.6465597545226531, 0.5759891811596606, 0.6081502195500234, 0.6290050670371272, 0.4657334139548547, 0.5613745590450681, 0.5140608861356697, 0.5338767128649504]


feature number: 6
test_rmse_ave: 0.00900434185133617
[0.009664085155991375, 0.010296317106475925, 0.008939112958483356, 0.008299875089102814, 0.008170259328825754, 0.007417464548915423, 0.009227415096791636, 0.008127094302488676, 0.009658909229686456, 0.010242885696600281]


test_r2_ave: 0.5205093877979063
[0.5016236113016971, 0.36367454752673356, 0.6061689933262695, 0.5658254990186502, 0.5856431712662866, 0.662963956289038, 0.49327969177072517, 0.6078043903253298, 0.42254908286224524,

In [18]:
fea_num=[5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.008850133051236346
[0.00955939458756484, 0.008760243455388136, 0.009224607556551865, 0.008152719217248444, 0.007736272817593002, 0.007292490596004614, 0.009293376690105576, 0.008730452254757961, 0.010621827327290327, 0.009129946009858685]


test_r2_ave: 0.5371021935874551
[0.5123629001823389, 0.5393746581463564, 0.5806111760360595, 0.581084748934835, 0.6284935393060298, 0.6742254563807313, 0.48600928212714367, 0.54740931344202, 0.30167533764346943, 0.5197755236755673]


feature number: 6
test_rmse_ave: 0.009078620280863654
[0.009127025714995022, 0.010657599541115767, 0.00858096627361468, 0.008547137882968148, 0.008297165397271229, 0.008185021681727083, 0.008708521547739502, 0.009197366416750744, 0.009815551433929174, 0.009669846918525191]


test_r2_ave: 0.5123990609502326
[0.5554767193535295, 0.31823567623925364, 0.6370945877832165, 0.539571051521546, 0.5726710531539989, 0.5896021679768908, 0.5486670341582788, 0.49770470839620584, 0.403667729040573, 0

In [19]:
fea_num=[5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.008688172590537366
[0.009252510495603277, 0.00944737434684313, 0.008809572131297561, 0.008096214463532559, 0.007909209320129004, 0.0074056216029698375, 0.009179918482849575, 0.00847272359076024, 0.009040357348249197, 0.00926822412313928]


test_r2_ave: 0.5559038012313698
[0.5431694539412513, 0.4642801671241248, 0.6175006629699364, 0.5868714500758203, 0.6116985999433646, 0.6640393407811833, 0.49848278642781907, 0.5737364411807572, 0.49414027776257163, 0.5051188321068693]


feature number: 6
test_rmse_ave: 0.00851677206382392
[0.008934138143295385, 0.009019872376837474, 0.008450362582761892, 0.007829402662616709, 0.007764646242331379, 0.007330124341672948, 0.009088144363610927, 0.008496591470635447, 0.008959193682985027, 0.009295244771492009]


test_r2_ave: 0.5729264765396002
[0.5740669997819383, 0.5116667888007962, 0.6480574773692991, 0.6136521859144983, 0.6257634803525811, 0.6708543855412468, 0.5084602675957259, 0.5713314677320229, 0.5031826468410999,

In [20]:
fea_num=[5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.008666501004116478
[0.009209460816245936, 0.009534067658771805, 0.008362192234921054, 0.008477147410077558, 0.00829779641993221, 0.007453726221873441, 0.008885819945178973, 0.008344749711948183, 0.008603330831263105, 0.00949671879095252]


test_r2_ave: 0.5575426224027261
[0.5474106070924236, 0.4544030484278553, 0.6553634379149712, 0.5470808626976289, 0.5726060515564118, 0.6596605729721375, 0.5301024159709418, 0.5865159514070377, 0.5418663911491359, 0.4804168848387168]


feature number: 6
test_rmse_ave: 0.008654147947832883
[0.009070130056673474, 0.008479499152282924, 0.009082378075732029, 0.008123633879639039, 0.008345083582558598, 0.007504338649876602, 0.008719381736782902, 0.008333483728091364, 0.00896818391386169, 0.009915366702830202]


test_r2_ave: 0.5600637907208192
[0.5610015453202088, 0.5684253985576757, 0.593444156126635, 0.584068430101925, 0.5677209392184203, 0.6550229229266041, 0.5475406391407882, 0.5876316613472976, 0.502185069636158, 0.43

In [ ]:
fea_num=[5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.008929185097473025
[0.010574995738842853, 0.009197091788330511, 0.008721569755514202, 0.008827011526657906, 0.008778069368345786, 0.007642710329729345, 0.009091807370018162, 0.008321250017905724, 0.008658066581356682, 0.009479278498029101]


test_r2_ave: 0.5308692691802874
[0.40324459028555426, 0.4922890646178376, 0.6251043749270927, 0.5089241371548463, 0.5216995003405859, 0.6421836440327511, 0.5080639543400095, 0.5888415015492373, 0.5360184104388652, 0.48232351411609375]


feature number: 6
test_rmse_ave: 0.008919064276853725
[0.009391699539033557, 0.010065317252733548, 0.008373964727777653, 0.008004780192564714, 0.008039098807615106, 0.008042903199391457, 0.008737607775328822, 0.009305802419027743, 0.009368220248718813, 0.00986124860634585]


test_r2_ave: 0.5302325009540658
[0.5293215194775933, 0.39190642855936175, 0.6543923797798261, 0.5961500599168562, 0.5988400665066911, 0.6037301111479235, 0.54564711943181, 0.4857908690574966, 0.45678332742809, 

In [ ]:
fea_num=[5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A6_019.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')